# Google Spanner

> [Spanner](https://cloud.google.com/spanner) is a highly scalable database that combines unlimited scalability with relational semantics, such as secondary indexes, strong consistency, schemas, and SQL providing 99.999% availability in one easy solution.

This notebook goes over how to use `Spanner` for GraphRAG with `SpannerGraphStore` and `SpannerGraphQAChain` class.

Learn more about the package on [GitHub](https://github.com/googleapis/langchain-google-spanner-python/).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-spanner-python/blob/main/docs/graph_store.ipynb)

## Before You Begin

To run this notebook, you will need to do the following:

 * [Create a Google Cloud Project](https://developers.google.com/workspace/guides/create-project)
 * [Enable the Cloud Spanner API](https://console.cloud.google.com/flows/enableapi?apiid=spanner.googleapis.com)
 * [Create a Spanner instance](https://cloud.google.com/spanner/docs/create-manage-instances)
 * [Create a Spanner database](https://cloud.google.com/spanner/docs/create-manage-databases)

### 🦜🔗 Library Installation
The integration lives in its own `langchain-google-spanner` package, so we need to install it.

In [ ]:
%pip install --upgrade --quiet langchain-google-spanner langchain-google-vertexai langchain-experimental json-repair pyvis

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 🔐 Authentication
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

* If you are using Colab to run this notebook, use the cell below and continue.
* If you are using Vertex AI Workbench, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [1]:
from google.colab import auth

auth.authenticate_user()

### ☁ Set Your Google Cloud Project
Set your Google Cloud project so that you can leverage Google Cloud resources within this notebook.

If you don't know your project ID, try the following:

* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "google.com:cloud-spanner-demo"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}
%env GOOGLE_CLOUD_PROJECT={PROJECT_ID}

### 💡 API Enablement
The `langchain-google-spanner` package requires that you [enable the Spanner API](https://console.cloud.google.com/flows/enableapi?apiid=spanner.googleapis.com) in your Google Cloud Project.

In [3]:
# enable Spanner API
!gcloud services enable spanner.googleapis.com

You must also and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

## Basic Usage

### Set Spanner database values
Find your database values, in the [Spanner Instances page](https://console.cloud.google.com/spanner?_ga=2.223735448.2062268965.1707700487-2088871159.1707257687).

In [15]:
# @title Set Your Values Here { display-mode: "form" }
INSTANCE = ""  # @param {type: "string"}
DATABASE = ""  # @param {type: "string"}
GRAPH_NAME = ""  # @param {type: "string"}

### SpannerGraphStore

To initialize the `SpannerGraphStore` class you need to provide 3 required arguments and other arguments are optional and only need to pass if it's different from default ones

1.   a Spanner instance id;
2.   a Spanner database id belongs to the above instance id;
3.   a Spanner graph name used to create a graph in the above database.

In [16]:
from langchain_google_spanner import SpannerGraphStore

graph_store = SpannerGraphStore(
    instance_id=INSTANCE,
    database_id=DATABASE,
    graph_name=GRAPH_NAME,
)

#### Add Graph Documents to Spanner Graph

In [ ]:
 # @title Extract Nodes and Edges from text snippets
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_google_vertexai import ChatVertexAI
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_snippets = [
# Text snippet for students graduting from Veritas University, Computer Science Dept 2017

"""
This was the graduation ceremony of 2017. A wave of jubilant graduates poured out of the
grand halls of Veritas University, their laughter echoing across the quad. Among them were
a cohort of exceptional students from the Computer Science department, a group that had
become known for their collaborative spirit and innovative ideas.
Leading the pack was Emily Davis, a coding whiz with a passion for cybersecurity, already
fielding offers from top tech firms. Beside her walked James Rodriguez, a quiet but
brilliant mind fascinated by artificial intelligence, dreaming of building machines that
could understand human emotions.  Trailing slightly behind, deep in conversation, were
Sarah Chen and Michael Patel, both aspiring game developers, eager to bring their creative
visions to life.  And then there was  Aisha Khan, a social justice advocate who planned to
use her coding skills to address inequality through technology.
As they celebrated their achievements, these Veritas University Computer Science graduates
were ready to embark on diverse paths, each carrying the potential to shape the future of
technology in their own unique way.
""",

# Text snippet for students graduting from Oakhaven University, Computer Science Dept 2016
"""
The year was 2016, and a palpable buzz filled the air as the graduating class of Oakhaven
university from Computer science and Engineering department emerged from the Beckman
Auditorium. Among them was a group of exceptional students, renowned for their
intellectual curiosity and groundbreaking research.
At the forefront was Alice Johnson, a gifted programmer with a fascination for quantum
computing, already collaborating with leading researchers in the field.  Beside her
strode David Kim, a brilliant theorist captivated by the intricacies of cryptography,
eager to contribute to the development of secure communication systems.  Engaged in an
animated discussion were Maria Rodriguez and Robert Lee, both passionate about robotics
and determined to push the boundaries of artificial intelligence.  And then there was
Chloe Brown, a visionary with a deep interest in bioinformatics, driven to unlock the
secrets of the human genome through computational analysis.
As they celebrated their accomplishments, these graduates, armed with their exceptional
skills and unwavering determination, were poised to make significant contributions to the world of computing and beyond.
""",

# Text snippet mentions the company Emily Davis founded.
# The snippet doesn't mention that she is an alumni of Veritas University
"""
Emily Davis, a name synonymous with cybersecurity innovation, turned that passion into a
thriving business.  In the year 2022, Davis founded Ironclad Security, a company that's
rapidly changing the landscape of cybersecurity solutions.
""",

# Text snippet mentions the company Alice Johnson founded.
# The snippet doesn't mention that she is an alumni of Oakhaven University.
"""
Alice Johnson had a vision that extended far beyond the classroom. Driven by an insatiable
curiosity about the potential of quantum mechanics, she founded Entangled Solutions, a
company poised to revolutionize industries through the power of quantum technology.
Entangled Solutions distinguishes itself by focusing on practical applications of quantum
computing.
"""
]

# Create splits for documents
documents = [Document(page_content=t) for t in text_snippets]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)
llm = ChatVertexAI(model="gemini-1.5-flash", temperature=0)
llm_transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes = ["College", "Deparatment", "Person", "Year", "Company"],
    allowed_relationships = ["AlumniOf", "StudiedInDepartment", "PartOf", "GraduatedInYear", "Founded"],
    node_properties=[ "description", ],
)
graph_documents = llm_transformer.convert_to_graph_documents(splits)


In [13]:
# @title Print extracted nodes and edges
for doc in graph_documents:
    print(doc.source.page_content[:100])
    print(doc.nodes)
    print(doc.relationships)
    print()

This was the graduation ceremony of 2017. A wave of jubilant graduates poured out of the
grand halls
[Node(id='Veritas University', type='College', properties={'description': 'grand halls'}), Node(id='Computer Science', type='Deparatment', properties={}), Node(id='2017', type='Year', properties={}), Node(id='Emily Davis', type='Person', properties={'description': 'coding whiz with a passion for cybersecurity'}), Node(id='James Rodriguez', type='Person', properties={'description': 'quiet but brilliant mind fascinated by artificial intelligence'}), Node(id='Sarah Chen', type='Person', properties={'description': 'aspiring game developers'}), Node(id='Michael Patel', type='Person', properties={'description': 'aspiring game developers'}), Node(id='Aisha Khan', type='Person', properties={'description': 'social justice advocate'})]
[Relationship(source=Node(id='Emily Davis', type='Person', properties={}), target=Node(id='Veritas University', type='College', properties={}), type='ALUMNIOF', pr

In [18]:
# @title Load the graph to Spanner Graph database
# Uncomment the line below, if you want to cleanup from
# previous iterations.
# BeWARE - THIS COULD REMOVE DATA FROM YOUR DATABASE !!!
# graph_store.cleanup()


for graph_document in graph_documents:
    graph_store.add_graph_documents([graph_document])

Waiting for DDL operations to complete...
Insert nodes of type `College`...
Insert nodes of type `Deparatment`...
Insert nodes of type `Year`...
Insert nodes of type `Person`...
Insert edges of type `Person_ALUMNIOF_College`...
Insert edges of type `Person_STUDIEDINDEPARTMENT_Deparatment`...
Insert edges of type `Person_GRADUATEDINYEAR_Year`...
No schema change required...
Insert nodes of type `College`...
Insert nodes of type `Deparatment`...
Insert nodes of type `Person`...
Insert edges of type `Person_ALUMNIOF_College`...
Insert edges of type `Person_STUDIEDINDEPARTMENT_Deparatment`...
Waiting for DDL operations to complete...
Insert nodes of type `College`...
Insert nodes of type `Deparatment`...
Insert nodes of type `Year`...
Insert nodes of type `Person`...
Insert edges of type `Person_ALUMNIOF_College`...
Insert edges of type `Deparatment_PARTOF_College`...
Insert edges of type `Person_STUDIEDINDEPARTMENT_Deparatment`...
Insert edges of type `College_GRADUATEDINYEAR_Year`...
Ins

### Initialize the Spanner Graph QA Chain
The Spanner Graph QA Chain takes two parameters, a SpannerGraphStore object and a language model.

In [30]:
from google.cloud import spanner
from langchain_google_vertexai import ChatVertexAI
from IPython.core.display import HTML

# Initialize llm object
llm = ChatVertexAI(model="gemini-1.5-flash-002", temperature=0)

# Initialize GraphQAChain
chain = SpannerGraphQAChain.from_llm(
  llm,
  graph=graph_store,
  allow_dangerous_requests=True,
  verbose=True,
  return_intermediate_steps=True
)

In [33]:
# @title Run Spanner Graph QA Chain 1
question =  "Who are the alumni of the college id Veritas University ?" # @param {type:"string"}
response = chain.invoke("query=" + question)
response["result"]



> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH graph_demo_2
MATCH (p:Person)-[:ALUMNIOF]->(c:College {id: "Veritas University"})
RETURN p.id AS person_id, c.id AS college_id
Full Context:
[{'person_id': 'Aisha Khan', 'college_id': 'Veritas University'}, {'person_id': 'Emily Davis', 'college_id': 'Veritas University'}, {'person_id': 'James Rodriguez', 'college_id': 'Veritas University'}, {'person_id': 'Michael Patel', 'college_id': 'Veritas University'}, {'person_id': 'Sarah Chen', 'college_id': 'Veritas University'}]

> Finished chain.


'Aisha Khan, Emily Davis, James Rodriguez, Michael Patel, and Sarah Chen are alumni of Veritas University.\n'

In [34]:
# @title Run Spanner Graph QA Chain 2
question =  "List the companies, their founders and the college they attended." # @param {type:"string"}
response = chain.invoke("query=" + question)
response["result"]



> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH graph_demo_2
MATCH (p:Person)-[:FOUNDED]->(c:Company), (p)-[:ALUMNIOF]->(cl:College)
RETURN c.id AS company_id, c.description AS company_description, p.id AS founder_id, p.description AS founder_description, cl.id AS college_id, cl.description AS college_description
Full Context:
[{'company_id': 'Entangled Solutions', 'company_description': 'Entangled Solutions distinguishes itself by focusing on practical applications of quantum computing.', 'founder_id': 'Alice Johnson', 'founder_description': 'Driven by an insatiable curiosity about the potential of quantum mechanics, she founded Entangled Solutions, a company poised to revolutionize industries through the power of quantum technology.', 'college_id': 'Oakhaven University', 'college_description': 'Oakhaven university'}, {'company_id': 'Ironclad Security', 'company_description': "a company that's rapidly changing the landscape of cybersecurity solutions", 'founder_id':

'Entangled Solutions, founded by Alice Johnson who attended Oakhaven University, focuses on practical applications of quantum computing.  Ironclad Security, founded by Emily Davis who attended Veritas University, is rapidly changing the landscape of cybersecurity solutions.\n'

In [35]:
# @title Run Spanner Graph QA Chain 3
question =  "Which companies were founded by alumni of college id Veritas University ? Who were the founders ?" # @param {type:"string"}
response = chain.invoke("query=" + question)
response["result"]



> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH graph_demo_2
MATCH (c:College {id: "Veritas University"})<-[:ALUMNIOF]-(p:Person)-[:FOUNDED]->(co:Company)
RETURN co.id AS company_id, co.description AS company_description, p.id AS founder_id, p.description AS founder_description
Full Context:
[{'company_id': 'Ironclad Security', 'company_description': "a company that's rapidly changing the landscape of cybersecurity solutions", 'founder_id': 'Emily Davis', 'founder_description': 'a name synonymous with cybersecurity innovation'}]

> Finished chain.


"Ironclad Security, a company that's rapidly changing the landscape of cybersecurity solutions, was founded by Emily Davis, a name synonymous with cybersecurity innovation.\n"

#### Clean up the graph

> USE IT WITH CAUTION!

Clean up all the nodes/edges in your graph and remove your graph definition.

In [ ]:
graph_store.cleanup()